# Evaluation notebook

## Setting up the training and testing dataframes

In [1]:
import pandas as pd

df_train = pd.read_csv("datasets/txt2sql_alerce_train_v3_1.csv")
df_test = pd.read_csv("datasets/txt2sql_alerce_test_v3_1.csv")

In [9]:
# Information about the dataframes
print(f"Shape of training set: {df_train.shape}")
df_train.head()

Shape of training set: (58, 12)


,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,13,Give me all the SNe that were first detected b...,"['object', 'probability']",\n-- mjd date for December = 59914.0\n-- mjd d...,\n-- Super Nova (SNe) is a large explosion tha...,"\nSELECT\n object.oid, probability.class_na...",simple,object,none,NaN,NaN,NaN
1,10,Get the object identifiers and probabilities i...,"['probability', 'object']",0,0,"\nSELECT\n sq1.oid, sq1.probability as SN_pro...",medium,object,simple,NaN,NaN,NaN
2,15,"Get the object identifiers, probabilities in t...","['object', 'probability', 'detection', 'magstat']",\n-- mjd date for September 01 = 60188.0\n-- m...,\n-- A fast riser is defined as an object whos...,"\nSELECT\n sq.oid, sq.probability, sq.candi...",advanced,other,multi,NaN,NaN,NaN
3,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,0,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,NaN,NaN,NaN
4,25,Query objects within 10 degress of the next po...,"['probability', 'object']",\n-- mjd date for February 01 = 59976.0\n-- mj...,0,"\nWITH catalog ( source_id, ra, dec) AS (\n ...",advanced,spatial,simple,NaN,NaN,NaN


In [10]:
print(f"Shape of testing set: {df_test.shape}")
df_test.head()

Shape of testing set: (52, 12)


,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,8,Query all objects that were first classified a...,"['probability', 'object']",\n-- mjd date for august 17 = 60173.0\n-- mjd ...,NaN,\nSELECT\n *\nFROM\n object\nINNER JOIN\...,simple,object,none,NaN,NaN,NaN
1,27,"Return the oids, meanra, meandec, ndet, firstm...","['probability', 'object']",\n-- object.ndet represents the total number o...,NaN,"\nSELECT\n object.oid, object.meanra, objec...",simple,object,simple,NaN,NaN,NaN
2,17,Return the objects classified as AGN that show...,"['probability', 'object', 'detection']",\n-- firstmjd represents the time of the first...,NaN,\nSELECT\n *\nFROM\n (\nSELECT *\nFROM (\nSE...,advanced,object,tree,NaN,NaN,NaN
3,37,"""Query the top 200 objects classified SNIa acc...",['probability'],NaN,NaN,\nSELECT\n *\nFROM\n probability\nWHERE\n ...,simple,object,none,NaN,NaN,NaN
4,34,Given this list of oids ['ZTF17aaadpsi' 'ZTF19...,['feature'],\n-- feature.name can be 'Amplitude'\n-- featu...,\n-- Multiband_period: Period obtained using t...,\nSELECT\n *\nFROM\n feature\nWHERE\n ...,simple,other,none,NaN,NaN,NaN


## Running experiments

I need the predictions CSV -> Talk to Jorge to see how to run it efficiently

In [ ]:
# Experiment and save paths
experiment_path = ""
save_path = ""

# Running parameters
ncpus = 2
db_min = 2

# Model to use
model = "gpt-4o"

# RAG parameters
max_tokens = 10000
size = 700
overlap = 300
quantity = 10

# Type of pipeline
format = "python"
self_corr = True
rag_pipe = True
direct = False

In [ ]:
# Running the evaluation in training
from pipeline.eval import run_eval_fcn

run_eval_fcn(df_train, experiment_path, save_path, model, max_tokens, format,
             ncpus, db_min, self_corr, rag_pipe, direct, size, overlap, 
             quantity)